## 앙상블

### Ensemble 의 종류
| 앙상블 유형          | 설명                                                                                         | 대표적인 예시                             |
|----------------------|----------------------------------------------------------------------------------------------|--------------------------------------------|
| 보팅(Voting)         | 다양한 기본 모델의 예측을 모아서 다수결 또는 가중 평균 등의 방식으로 결합한다.             | - 다수결 투표<br/>- 평균                    |
| 배깅(Bagging)        | 동일한 모델을 여러 번 학습하여 예측 결과를 평균화한다.                                    | - 랜덤 포레스트                           |
| 부스팅(Boosting)     | 약한 모델을 순차적으로 학습하여 강한 모델을 생성한다.                                    | - AdaBoost<br/>- Gradient Boosting<br/>- XGBoost<br/>- LightGBM |
| 스태킹(Stacking)     | 여러 다른 모델을 조합하여 메타 모델을 생성한다.                                        | - 스태킹                                   |

Voting VS Bagging
- Voting : 위의 설명과 같음
- Bagging : 각 모델들이 부트스트랩 sample(복원추출된 random sub dataset)으로 학습함.

In [1]:
import sklearn.ensemble as skens
import sklearn.tree as sktree
import pandas as pd

In [4]:
Data=pd.read_csv('C:/Users/User/Desktop/glass.csv')
data=pd.DataFrame(Data)
target=["Type"]
feature=list(data.columns.difference(target))
data['Type']=data["Type"].apply(str)
import numpy as np
np.random.seed(10)
import sklearn.model_selection as skmod
train,test=skmod.train_test_split(data,train_size=0.8,random_state=10)

모델평가를 위해서 검증용 데이터도 추가 생성해야한다.

In [6]:
sub_train,sub_test=skmod.train_test_split(train,train_size=0.8,random_state=10)

### 0. 기본 RandomForest

참고
> min_impurity_decrease : 불순도 임계값         
> min_weight_fraction_leaf : 분리하기 전 노드에서 고려해야하는 샘플의 가중치 비율(낮을수록 쉽게 분할)           
> n_jobs : 사용할 CPU 개수


In [7]:
rf_model=skens.RandomForestClassifier(n_estimators=100,max_depth=5,criterion='gini',max_features='sqrt',
                                      min_impurity_decrease=0,max_leaf_nodes=None,min_samples_leaf=1,
                                      min_samples_split=2,min_weight_fraction_leaf=0,
                                      n_jobs=None,warm_start=False,verbose=0)

In [8]:
rf_model.fit(X=sub_train[feature],y=sub_train[target])

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=5, min_impurity_decrease=0,
                       min_weight_fraction_leaf=0)

In [9]:
import sklearn.metrics as skmet

### 모델평가

성능평가
-1. 훈련 검증

In [10]:
skmet.accuracy_score(sub_test[target],rf_model.predict(sub_test[feature]))

0.6857142857142857

성능평가
-2. 실제 검증

In [11]:
skmet.accuracy_score(test[target],rf_model.predict(test[feature]))

0.6046511627906976

과적합은 일어나지 않았으며, `accuaracy`가 최종 60% 나왔다

In [20]:
# feature_importance
pd.DataFrame(list(zip(rf_model.feature_names_in_,rf_model.feature_importances_)))

,0,1
0,Al,0.181405
1,Ba,0.094857
2,Ca,0.127130
3,Fe,0.019376
4,K,0.078423
5,Mg,0.196259
6,Na,0.113732
7,RI,0.128131
8,Si,0.060687


### 1. ExtraTree를 이용한 Classification
일반적인 decision tree보다 더 높은 무작위성을 강조한 모델 extratree를 이용

In [15]:
ex_rf_model=skens.ExtraTreesClassifier(n_estimators=100,max_depth=5,criterion='gini',max_features='sqrt',
                                      min_impurity_decrease=0,max_leaf_nodes=None,min_samples_leaf=1,
                                      min_samples_split=2,min_weight_fraction_leaf=0,
                                      n_jobs=None,warm_start=False,verbose=0)

In [16]:
ex_rf_model.fit(X=sub_train[feature],y=sub_train[target])

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


ExtraTreesClassifier(max_depth=5, min_impurity_decrease=0,
                     min_weight_fraction_leaf=0)

#### 모델평가

성능평가 -1. 훈련검증

In [21]:
skmet.accuracy_score(sub_test[target],ex_rf_model.predict(sub_test[feature]))

0.6571428571428571

성능평가 -2. 실제 검증

In [22]:
skmet.accuracy_score(test[target],ex_rf_model.predict(test[feature]))

0.5581395348837209

In [25]:
# feature importance
pd.DataFrame(list(zip(ex_rf_model.feature_names_in_,ex_rf_model.feature_importances_)))

,0,1
0,Al,0.149258
1,Ba,0.156580
2,Ca,0.114328
3,Fe,0.031209
4,K,0.077086
5,Mg,0.237320
6,Na,0.086114
7,RI,0.106616
8,Si,0.041489


기존 randomforest보다 더 낮은 성능을 보임           
> 무작위성을 증가시켰지만 성능이 떨어지는것을 보아 depth 등 가지치기와 같은 억제력을 감소시켜 비교해야 할듯